# Personality Discovery

## Imports and DataFrames

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [61]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [62]:
df_train.sample(5)

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
6793,6793,2.0,No,6.0,5.0,No,8.0,5.0,Extrovert
10669,10669,1.0,No,4.0,4.0,No,6.0,4.0,Introvert
13177,13177,1.0,No,5.0,6.0,NaN,10.0,3.0,Extrovert
264,264,2.0,No,9.0,6.0,No,4.0,3.0,Extrovert
3091,3091,2.0,No,5.0,NaN,No,6.0,3.0,Extrovert


## Preprocessing

### Train

In [63]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [64]:
def ValuesColumnEqualExtrovert(df,column_name):
    """
    Returns a dictionary with counts of each unique value in `column_name`,
    but only for rows where Personality == 'Extrovert'.
    
    Parameters:
    - df: pd.DataFrame — the dataset
    - column_name: str — the column to analyze
    
    Returns:
    - dict: {unique_value: count}
    """
    # Filter to only 'Extrovert' rows
    extroverts = df[df['Personality'] == 'Extrovert']
    
    # Get value counts
    value_counts = extroverts[column_name].value_counts()
    
    # Convert to dictionary
    value_dict = dict(value_counts)
    
    return value_dict

def ValuesColumnEqualIntrovert(df,column_name):
    """
    Returns a dictionary with counts of each unique value in `column_name`,
    but only for rows where Personality == 'Introvert'.
    
    Parameters:
    - df: pd.DataFrame — the dataset
    - column_name: str — the column to analyze
    
    Returns:
    - dict: {unique_value: count}
    """
    # Filter to only 'Introvert' rows
    introverts = df[df['Personality'] == 'Introvert']
    
    # Get value counts
    value_counts = introverts[column_name].value_counts()

    # Convert to dictionary
    value_dict = dict(value_counts)
    
    return value_dict

In [65]:
values_ext_TSA = ValuesColumnEqualExtrovert(df_train,'Time_spent_Alone')
values_int_TSA = ValuesColumnEqualIntrovert(df_train,'Time_spent_Alone')

In [78]:
print(values_ext_TSA)
for key,val in values_ext_TSA.items():
    if val > 1000:
        df_train[df_train['Time_spent_Alone'] == key]['Personality']

{0.0: 3061, 3.0: 3005, 2.0: 2971, 1.0: 2903, 4.0: 550, 5.0: 58, 6.0: 56, 9.0: 42, 7.0: 40, 8.0: 36, 11.0: 34, 10.0: 34}


KeyError: "None of [Index([0.0, 1.0, 6.0, 3.0, 1.0, 2.0, 1.0, 2.0, 4.0, 1.0,\n       ...\n       6.0, nan, nan, 3.0, 3.0, 3.0, 1.0, 7.0, nan, 1.0],\n      dtype='float64', length=18524)] are in the [columns]"

In [67]:
values_int_TSA

{5.0: 575,
 10.0: 553,
 8.0: 546,
 7.0: 541,
 9.0: 532,
 4.0: 529,
 6.0: 518,
 11.0: 458,
 0.0: 78,
 3.0: 76,
 1.0: 70,
 2.0: 68}

### Test